In [1]:
from speechbrain.lobes.augment import TimeDomainSpecAugment
from speechbrain.lobes.augment import EnvCorrupt
from speechbrain.dataio.dataio import read_audio
import numpy as np
import random
import torch
import torchaudio
import pandas as pd

The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.


In [2]:
#!pip install speechbrain

In [3]:


def corrupter(audio_file,save_name):

    signal = read_audio(audio_file)
    clean = signal.unsqueeze(0)
    do_augment = TimeDomainSpecAugment(speeds=[i for i in range(85, 116)],
                                    perturb_prob=1, 
                                    drop_freq_prob=1, 
                                    drop_chunk_prob=1,
                                    drop_chunk_length_low=1000,
                                    drop_freq_count_low=0,
                                    drop_freq_count_high=np.random.randint(50, size=1)[0],
                                    drop_chunk_count_low=0,
                                    drop_chunk_count_high=np.random.randint(50, size=1)[0],
                                    drop_chunk_length_high=5000)
    do_augment_env = EnvCorrupt(reverb_prob=1.0,
                              noise_prob=1.0,
                              noise_num_workers=np.random.randint(100, size=1)[0]
                              )
    augmented_signal = do_augment(clean, torch.ones(1))
    augmented_signal_env = do_augment_env(augmented_signal, torch.ones(1))
    torchaudio.save(save_name, augmented_signal_env, sample_rate=16000)

In [4]:
tab=pd.read_csv('SPEAKERS_TYPE.csv',';')

C:\Users\Paul\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
tab.sort_values(by='ID')

,ID,Name,ORIGIN,SEX
130,19,LibriSpeech/19-198-0000.flac,TRAIN,F
168,26,LibriSpeech/26-495-0000.flac,TRAIN,M
234,27,LibriSpeech/27-123349-0000.flac,TEST,M
232,32,LibriSpeech/32-21625-0000.flac,TEST,F
28,39,LibriSpeech/39-121914-0000.flac,TRAIN,F
...,...,...,...,...
233,8747,LibriSpeech/8747-293952-0000.flac,TEST,M
90,8770,LibriSpeech/8770-295462-0000.flac,TRAIN,M
248,8797,LibriSpeech/8797-294123-0000.flac,TEST,M
196,8838,LibriSpeech/8838-298545-0000.flac,TRAIN,M


In [6]:
n0=tab['ID'].max()+1
for i in range(len(tab['Name'])):
    save_n='LibriSpeech/'+str(n0+i)+'.flac'
    corrupter(tab['Name'].iloc[i],save_n)
    tab=tab.append([n0+i,save_n,'TRAIN',tab['SEX'].iloc[i]])
    

In [7]:
tab.to_csv('SPEAKERS_TYPE_extended.csv',';',index=False)